## **Hands-on Lab: Interactive Visual Analytics with Folium**


Let's first import required Python packages for this lab:


In [1]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [17]:
## Task 1: Mark all launch sites on a map
launch_map = folium.Map(location=[28.5, -80.5], zoom_start=5)
launch_map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
spacex_df=pd.read_csv("spacex_launch_geo.csv")

Now, you can take a look at what are the coordinates for each site.


In [5]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [24]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
folium.Circle(
    nasa_coordinate,
    radius=1000,
    color='#ffff00',
    popup='NASA Johnson Space Center').add_to(site_map)

for index, site in launch_sites_df.iterrows():
    site_coordinate = [site['Lat'], site['Long']]
    
    folium.Circle(
        site_coordinate,
        radius=1000,
        color='#000000',).add_child(folium.Popup(site['Launch Site'])).add_to(site_map)
    
    folium.map.Marker(
        site_coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{site["Launch Site"]}</b></div>')).add_to(site_map)
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [20]:
world_map = folium.Map(location=[0, 0], zoom_start=2)

# Add equator line
folium.PolyLine(
    locations=[(0, -180), (0, 180)],
    color='red',
    weight=2,
    popup='Equator'
).add_to(world_map)

# Add launch sites
for index, site in launch_sites_df.iterrows():
    folium.Marker(
        location=[site['Lat'], site['Long']],
        popup=site['Launch Site']
    ).add_to(world_map)

world_map

In [30]:
# Task 2: Mark the success/failed launches for each site on the map
outcome_map = folium.Map(location=[28.5, -80.5], zoom_start=3.5)

# Color mapping
color_dict = {1: 'green', 0: 'red'}

# Add markers for each launch
for index, launch in spacex_df.iterrows():
    folium.CircleMarker(
        location=[launch['Lat'], launch['Long']],
        radius=5,
        color=color_dict[launch['class']],
        fill=True,
        popup=f"{launch['Launch Site']}<br>Outcome: {'Success' if launch['class'] else 'Failure'}"
    ).add_to(outcome_map)

# Add legend
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 120px; height: 100px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white">
     <b>Launch Outcomes</b><br>
     <i class="fa fa-circle fa-1x" style="color:green"></i> Success<br>
     <i class="fa fa-circle fa-1x" style="color:red"></i> Failure
     </div>
     '''
outcome_map.get_root().html.add_child(folium.Element(legend_html))

outcome_map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [31]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [36]:
launch_records_map = folium.Map(location=[28.57, -80.65], zoom_start=3)
marker_cluster = MarkerCluster().add_to(launch_records_map)
color_mapping = {1: 'green', 0: 'red'}
for index, record in spacex_df.iterrows():
    folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color=color_mapping[record['class']], icon='rocket', prefix='fa'),
        popup=f"""
        <b>Launch Site:</b> {record['Launch Site']}<br>
        <b>Outcome:</b> {'Success' if record['class'] else 'Failure'}<br>
        <b>Coordinates:</b> {record['Lat']:.4f}°N, {record['Long']:.4f}°W
        """
    ).add_to(marker_cluster)

legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 150px; height: 100px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white; padding: 5px">
<b>Launch Outcomes</b><br>
<i class="fa fa-rocket" style="color:green"></i> Success<br>
<i class="fa fa-rocket" style="color:red"></i> Failure<br>
<i class="fa fa-circle"></i> Cluster of markers
</div>
'''
launch_records_map.get_root().html.add_child(folium.Element(legend_html))

launch_records_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
